In [ ]:
!pip install python-dotenv
!pip install google-generativeai
!pip install langchain-google-genai
!pip install langchain

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #find_dotenv() is used to find the .env file
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

### Jaka jest róznica miedzy zwyklym modelem LLM a modelem typu Chat Model?
- LLM przyjmuje jeden tekst i zwraca tekst. Piszesz -> odpsiuje 
- Chat Model przyjmuje listę wiadomości. Kada wiadomośc ma swoją rolę (system, user, assistant) To jak rozmoa na czacie.

Stosuje się dwa rózne importy przez to:
- LLM `from langchain.llms import OpenAI`
- ChatModel `from langchain.chat_models import ChatOpenAI`

In [ ]:
def get_completion(prompt, model="gemini-2.5-flash"):
    model_instance = genai.GenerativeModel(model)
    response = model_instance.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(temperature=0)
    )
    return response.text


- `temperature` - to parametr "kreatywności":
    - 0 oznacza, ze model jest deterministyczny
    - wyzsze wartosci np. 0.7 sprawilby ze model bylby bardziej fantazyjny
    
- `response.text` - odpowiedz z Gemini API. Model zwraca obiekt response, z którego wyciagamy tekst przez atrybut `.text`

In [ ]:
get_completion("What is 1+1?")

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)
print(response)



### ChatAPI : LangChain - Gemini approach

In [ ]:
!pip install --upgrade langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0)
chat

Uywamy `ChatPromptTemplate` aby tworzyc formularz dla modelu
1. `template_string` - ot makieta, nawiasy klamrowe {style} i {text} - langchain rozpozna je jako puste pola do wypelnienia
2. `ChatPromptTemplate.from_template(...)` - metoda zamienia zwykły tekst w inteligentny obiekt LangChain. Ten obiekt wie jakie ma zmienne i jak przygotowac je do modelu
3. `prompt.format(...)` - wstrzykujemy konkretne dane do formularza. Operacja ta to wciaz tekst gotowy od wyslania

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}.
text: ```{text}```
"""



In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template_string)
prompt.format(style="American English", text="I'm very happy with the product")

print(type(prompt))
print(type(prompt[0]))
prompt

Dlaczego `prompt[0]` i typy sa takie wazne?

Kiedy sprawdzamy type(prompt) zobaczymy ze to nie jest zwykly `str` 

`type(prompt)` - to struktura, która moze skladac sie z wielu wiadomosci (System. Human, AI)



In [ ]:
prompt.messages[0].prompt.input_variables

Hierarchia obiektu `PromptTemplate`

`prompt` : `ChatPromptTemplate` - caly szablon-matka. zawiera liste wiadomosci, definicje zmiennych i instrukcje formatowania

`prompt.messages` : `list` - lista wszystkich czesci skladowych Twojego promptu. Nawet jesli masz tylko ejden tekst, on trafia na te liste

`prompt.messages[0]`: `HumanMessagePromptTemplate` - pierwszy klocek na liscie. Langchatin domyslnie zaklada, ze szablon tektstowy to wiadomosc od czlowieka



In [ ]:
template_string_1 = "Przetłumacz tekst: {text} na język {language} w stylu {style}" #dodanie 3 zmiennej
prompt_3_variables = ChatPromptTemplate.from_template(template_string_1)

prompt_3_variables.messages[0].prompt.input_variables

In [ ]:
customer_message = ChatPromptTemplate.from_template(template_string)
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_message = customer_message.format_messages(style=customer_style, text=customer_email)

In [ ]:
type(customer_message[0])

In [ ]:
customer_response = chat.invoke(customer_message)

In [ ]:
customer_response.content

Nastepny przyklad

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt.format_messages(style=service_style_pirate, text=service_reply)
service_response = chat.invoke(service_messages)
service_response.content


Krótkie powtorzenie do tego etapu:

1. Defincja szablonu 
Zamiast pisac gotowy tekst, tworzymy formularz z miejscami na zmienne w nawiacahc klamrowych
- Cel: oddzielenie instrukcji od danych wejsciowych
- Kod `template_string = "Przeltumacz {text} na styl {style}`

2. Utworzenie obiektu promptu
- Uzycie klasy `ChatPromptTemplate` aby zamienic suroy tekst w inteligetnych obiekt LangChain
- Cel: przygotowanie struktury, ktora rozumieja modele typu Chat
- Kod: `prompt_template = ChatPromptTemplate.from_template(template_string)`

3. Formatowanie - wstrzykuejsz konkretne wartosci w miejscach zmiennych. Najlepiej uzyc metody `format_messages`
- Cel: zamiana formularza na liste konkretnych wiadomosci np. `HumanMessage` ktore sa gotowe do wyslania
- Kod: `formatted_prompt = prompt_template.format_messages(sty;e=customer_style, text=customer_email)`

4. Wywolanie modelu invoke
- Kod: `response = chat.invoke(formatted_prompt)`

5. Wyciagniecie tresci
- Kod `print(response.content)`

### Output Parser

Jak powinnien wygladac output

In [ ]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
prompt_template_for_parser = ChatPromptTemplate.from_template(review_template)
messages = prompt_template_for_parser.format_messages(text=customer_review)
chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
response = chat.invoke(messages)







In [ ]:
print(response.content)

In [ ]:
#przy lini response.content.get() dostaniemy error ze wzgledu ze tos tr

### Parse the LLM output string into a Python dictionary

In [ ]:
#!pip install langchain